In [ ]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils, TopicAndPartition

In [ ]:
spark = SparkSession.builder \
    .appName('test') \
    .getOrCreate()
sc = spark.sparkContext
sql_ctx = SQLContext(sc)
ssc = StreamingContext(sc, 10)

处理函数

In [ ]:
def process_func(ts, rdd):
    # do something
    # offset commit
    return rdd

连接kafka

In [ ]:
kafka_params = {
    'bootstrap.servers': KAFKA_CONFIG['servers'],
    'group.id': KAFKA_CONFIG['group_id'],
    'auto.offset.reset': 'largest',
    'auto.commit.enable': 'false', # 测试了一下这个参数 是无效的 不会commit
}
dks = KafkaUtils.createDirectStream(ssc, [KAFKA_CONFIG['topics']], kafka_params) 

dks = dks.transform(process_func)
dks.pprint()

不能在`process_func`中捕获错误

而是捕获`ssc.awaitTermination()`的错误

In [ ]:
ssc.start()

try:
    ssc.awaitTermination()
except:
    # handle error
    ssc.stop()

sc.stop()
